In [1]:
import re
from collections import defaultdict

from scipy.sparse import csr_matrix, lil_matrix, vstack
from scipy.sparse.linalg import norm
import numpy as np
from sklearn.cluster import AgglomerativeClustering

## 1. Implementacja metryk

#### Funkcje pomocnicze zwracające macierz wektorów bag-of-words oraz zbiór stopwords jako zbiór najczęsciej występujących słów

In [2]:
def get_document_term_matrix(lines, stopwords=None):
    if stopwords is None:
        stopwords = set()

    words = dict()
    vectors = []
    word_idx = 0

    for i, line in enumerate(lines):
        tokens = list(filter(lambda x: x != "" and x not in stopwords, re.split(r'\W+', line.lower())))

        for token in tokens:
            if token not in words:
                words[token] = word_idx
                word_idx += 1

        vector = np.zeros(word_idx, dtype=np.int32)

        for token in tokens:
            vector[words[token]] += 1

        vector = csr_matrix(vector)
        vectors.append(vector)

    n = len(vectors)
    k = len(words)
    matrix = lil_matrix((n, k), dtype=np.float32)

    for i, vector in enumerate(vectors):
        vector.resize((1, k))
        if norm(vector) > 0:
            vector = vector / norm(vector)
        matrix[i] = vector

    matrix = matrix.tocsr()
    return matrix


def get_stopwords(lines, minimum_occurrences=300):
    word_occurrences = defaultdict(int)

    for line in lines:
        tokens = list(filter(lambda x: x != "", re.split(r'\W+', line.lower())))

        for token in tokens:
            word_occurrences[token] += 1

    return {x[0] for x in filter(lambda x: x[1] > minimum_occurrences, word_occurrences.items())}

#### Funkcja pomocnicza do klasteryzacji

In [3]:
def clusterize(lines, dist_function, threshold, use_stopwords=True):
    if use_stopwords:
        stopwords = get_stopwords(lines)
    else:
        stopwords = None

    X = get_document_term_matrix(lines, stopwords=stopwords)

    # dist matrix
    dist = dist_function(X)

    clustering = AgglomerativeClustering(n_clusters=None, affinity='precomputed', linkage='average', distance_threshold=threshold)
    clustering.fit(dist)

    return clustering.labels_

#### 1) metryka cosinusowa

In [4]:
def cosine_distance(X):
    return 1 - (X @ X.T).toarray()


def cosine_clusterize(lines, use_stopwords=True):
    return clusterize(lines, cosine_distance, 0.4, use_stopwords=use_stopwords)

#### 2) metryka euclidesowa

In [5]:
def euclidean_distance(X):
    n = X.shape[0]

    dist = np.zeros((n, n), dtype=np.float64)
    ones = csr_matrix(np.ones(n)).T

    for i in range(n):
        dist[i, :] = norm(X - (ones @ X[i]), axis=1) / np.sqrt(2)

    return dist


def euclidean_clusterize(lines, use_stopwords=True):
    return clusterize(lines, euclidean_distance, 0.6, use_stopwords=use_stopwords)

#### 3) metryka dice

In [6]:
def dice_distance(X):
    n = X.shape[0]

    X = (X > 0).astype(np.int32)

    sums = np.sum(X, axis=1)
    sums = np.repeat(sums, n, axis=1) + sums.T

    dist = (X @ X.T).toarray()
    dist *= 2

    return 1 - dist / sums


def dice_clusterize(lines, use_stopwords=True):
    return clusterize(lines, dice_distance, 0.5, use_stopwords=use_stopwords)

## 2. Implementacja indeksu Daviesa-Bouldina do oceny klasteryzacji

In [7]:
def davies_bouldin_score(X, labels):
    k = max(labels) + 1
    cluster_matrices = [[] for _ in range(k)]

    for i, label in enumerate(labels):
        cluster_matrices[label].append(X[i])

    for i in range(len(cluster_matrices)):
        cluster_matrices[i] = vstack(cluster_matrices[i])

    centroids = [np.sum(matrix, axis=0) / matrix.shape[0] for matrix in cluster_matrices]

    avg_dist = [0 for _ in cluster_matrices]
    for i, matrix in enumerate(cluster_matrices):
        size = matrix.shape[0]
        avg_dist[i] = np.sum(np.linalg.norm(matrix - (np.ones((size, 1)) @ centroids[i]), axis=1), axis=0) / size

    R = np.zeros((k, k))
    for i in range(k):
        for j in range(i + 1, k):
            R[i, j] = (avg_dist[i] + avg_dist[j]) / np.linalg.norm(centroids[i] - centroids[j])
            R[j, i] = R[i, j]

    return np.sum(np.max(R, axis=1)) / k

## 3. Tworzenie stoplisty

Generujemy stoplistę przy użyciu wcześniej zaimplementowanej funkcji $get\_stopwords$, która przyjmuje jako argument tekst do klasteryzacji jako $lines$ oraz opcjonalnie minimalną liczbę wystąpień słowa w tekście, aby zostało dodane do stoplisty (domyślnie 300).

#### Stoplista zawierająca słowa występujące ponad 300 razy

In [8]:
with open("./lines.txt") as file:
    file_lines = file.readlines()
    stopwords = get_stopwords(file_lines)
    print(stopwords)

{'street', 'no', 'b', 'finland', 'ltd', 'z', '1', 'city', '81', 'china', '48', 'f', 'office', 'as', '58', '495', 'ningbo', 'of', 'international', 'ul', 'ooo', 'st', 'petersburg', 'sp', 'moscow', '358', 'oy', 'tel', '2', 'and', 'shanghai', 'district', 'fax', 's', 'p', '86', 'road', 'gdynia', 'shenzhen', '22', 'llc', 'o', 'limited', '812', 'poland', 'logistics', '7', 'str', 'russia', '5', 'co', 'building', 'c', '3', 'a'}


#### Stoplista zawierająca słowa występujące ponad 100 razy

In [9]:
with open("./lines.txt") as file:
    file_lines = file.readlines()
    stopwords = get_stopwords(file_lines, minimum_occurrences=100)
    print(stopwords)

{'for', 'k', 'b', 'finland', 'ltd', 'trading', 'city', 'industry', 'f', 'phone', '58', 'guangdong', '495', 'cargo', '6', 'panalpina', 'petersburg', 'a', 'moscow', 'thailand', 'district', '18', 'attn', 's', '50', 'south', '62', 'o', '44', '13', 'logistics', 'str', 'intl', 'vantaa', '3', 'code', 'kong', 'jiangsu', '10', 'china', 'federation', 'm', 'warszawa', '11', 'ningbo', '00', 'agent', 'st', 'russian', 'polska', 'forwarding', 'and', 'order', 'shipping', 'town', 'fax', 'eori', '05', 'zone', '86', 'nagel', '8', 'shenzhen', 'centre', 'guangzhou', '39', 'the', 'e', 'saint', 'c', 'park', 'global', 'center', '31', '1', 'trade', '81', 'floor', 'on', 'rm', 'kotka', 'zip', '190020', 'kuehne', 'behalf', 'branch', 'international', 'forward', 'ul', 'rd', 'sp', 'd', '32', '2', '9', 'fmg', 'hong', 'box', 'east', 'inn', 'zhejiang', 'llc', 'freight', '812', 'mail', 'poland', 'korea', 'group', 'fi', 'room', 'business', 'gdynia', '22', 'com', 'street', 'plaza', 'lit', '4', 'pl', 'z', '12', '19', 'sea'

## 4. Klasteryzacja zawartości pliku lines.txt przy użyciu metryk z pkt. 1

In [10]:
with open("./lines.txt") as file:
    file_lines = file.readlines()
    n = len(file_lines)

    labels1 = cosine_clusterize(file_lines)
    labels2 = euclidean_clusterize(file_lines)
    labels3 = dice_clusterize(file_lines)

    print("Klastry przypisane przy użyciu danej metryki dla danej lini pliku lines.txt (klasteryzacja przebiega dla całego pliku, wyświetalne jest pierwsze 30): \n")
    for i in range(30):
        print(f"COSINE_LABEL: {labels1[i]},  EUCLIDEAN_LABEL: {labels2[i]},  DICE_LABEL:{labels3[i]}")
        print(file_lines[i])

    clusters1 = [[] for _ in range(max(labels1) + 1)]
    clusters2 = [[] for _ in range(max(labels2) + 1)]
    clusters3 = [[] for _ in range(max(labels3) + 1)]

    for labels, clusters in [(labels1, clusters1), (labels2, clusters2), (labels3, clusters3)]:
        for i, label in enumerate(labels):
            clusters[label].append(file_lines[i])

    for filename, clusters in [(f"result_cosine.txt", clusters1), (f"result_euclidean.txt", clusters2), (f"result_dice.txt", clusters3)]:
        with open(f"./{filename}", "w") as result:
            for cluster in clusters:
                result.write("##########\n")
                for item in cluster:
                    result.write(item)
                result.write("\n")

Klastry przypisane przy użyciu danej metryki dla danej lini pliku lines.txt (klasteryzacja przebiega dla całego pliku, wyświetalne jest pierwsze 30): 

COSINE_LABEL: 2694,  EUCLIDEAN_LABEL: 2684,  DICE_LABEL:2682
/11692589 RD TUNA CANNERS, LTD. PORTION 1004, SIAR NORTH COAST ROAD, P.O.BOX 2113, MADANG, PAPUA NEW GUINEA

COSINE_LABEL: 2513,  EUCLIDEAN_LABEL: 2487,  DICE_LABEL:2687
''PA INTERIOR'' LTD BOLSHAYA LUBYANKA STREET, 16/4 MOSCOW, 101000, RUSSIA INN/KPP 7704550148//770801001 495-984-8611

COSINE_LABEL: 429,  EUCLIDEAN_LABEL: 1712,  DICE_LABEL:1586
''SSONTEX''  Sp.ZO.O.IMPORT-EXPORTUL:PRZECLAWSKA 5 03-879 WARSZAWA,POLAND NIP 113-01-17-669

COSINE_LABEL: 429,  EUCLIDEAN_LABEL: 1712,  DICE_LABEL:1586
''SSONTEX''SP.ZO.O.IMPORT-EXPORT UL:PRZECLAWSKA 5 03-879 WARSZAWA,POLAND NIP 113-01-17-669 TEL./FAX.:0048(022)217 6532--

COSINE_LABEL: 262,  EUCLIDEAN_LABEL: 298,  DICE_LABEL:148
''TOPEX SP. Z O.O.'' SPOLKA KOMANDYTOWA UL. POGRANICZNA 2/4  02-285 WARSZAWA POLAND

COSINE_LABEL: 3583,  

Powyższy fragment kodu dokonuje klasteryzacji dla całego pliku przy użyciu wszystkich 3 zaimplementowanych metryk, wyświetla pierwsze 30 lini z przypisanymi numerami klastra oraz zapisuje do trzech oddzielnych plików podział na klastry. Początkowe klastry z tych plików wyglądają następująco:

#### Klasteryzacja przy użyciu metryki cosinusowej
Pierwsze 100 lini z pliku result_cosine.txt

In [11]:
with open("result_cosine.txt", "r") as file:
    for _ in range(100):
        print(file.readline(), end="")

##########
RAHTIHUOLINTA SUOMI OY RAHTITIE 2 01530 VANTAA FINLAND EORI NO.:FI0871169-1 TEL:+35895868820 FAX:+358958688282 FI-
RAHTIHUOLINTA SUOMI OY RAHTITIE 2 01530 VANTAA FINLAND TEL:+35895868820 FAX:+358958688282 EORI NO.:FI0871169-1 FI-
RAHTIHUOLINTA SUOMI OY RAHTITIE 2 FI-01530 VANTAA FINLAND AS AGENT OFPELORUS OCEAN LINE

##########
ITALUX SP. Z O.O. SP. KOMADYTOWA 04-697 WARSZAWA,UL. MROWCZA 208,POLAND TEL:0048-022-8156962 FAX:0048-022-8156963
ITALUX SP.Z O.O. SP. KOMADYTOWA  04-697 WARSZAWA UL MROWCZA 208 +48 228156962

##########
SCAN GLOBAL LOGISTICS  (FINLAND) OYKOIVUHAANTIE 2-4 D 01510 VANTAA FINLAND
SCAN GLOBAL LOGISTICS (FINLAND) OYKOIVUHAANTIE 2-4 D 01510 VANTAA FINLAND CONTACT:KALLE PAKKI
SCAN GLOBAL LOGISTICS(FINLAND)OY KOIVUHAANTIE 2-4D FI-01510 VANTAA FINLAND
SCAN GLOBAL LOGISTICS (FINLAND) OYKOIVUHAANTIE 2-4D FI-01510 VANTAA TEL:+358 5 2604450 FAX:+358 5 2604459
SCAN GLOBAL LOGISTICS (FINLAND) OYKOIVUHAANTIE VANTAA(VANDA) 01510 FINLAND
SCAN GLOBAL LOGISTICS (FINLAND

#### Klasteryzacja przy użyciu metryki euklidesowej
Pierwsze 100 lini z pliku result_euclidean.txt

In [12]:
with open("result_euclidean.txt", "r") as file:
    for _ in range(100):
        print(file.readline(), end="")

##########
Saudi Basic Industries Corporation(SABIC) P.O. Box 5101 Riyadh 11422Kingdom of Saudi Arabia
SAUDI BASIC INDUSTRIES CORPORATION(SABIC) P.O. BOX 5101 RIYADH 11422KINGDOM OF SAUDI ARABIA
SAUDI BASIC INDUSTRIES CORPORATIONKINGDOM OF SAUDI ARABIAPO BOX 5101RIYADH 11422SAUDI ARABIA

##########
TAKERNG PINEAPPLE INDUSTRIAL CO., LTD. 220 MOO 6, T.SALALAI , SAMROIYOD, PRACHUABKIRIKHAN 77180 , THAILAND
TAKERNG PINEAPPLE INDUSTRIAL CO., LTD. 220 MOO 6, T. SALALAI , SAMROIYOD, PRACHUABKIRIKHAN 77180 , THAILAND
TAKERNG PINEAPPLE INDUSTRIAL CO LTD-220 MOO 6 PHETKASEM ROADSALALAI SAMROIYOD, PRACHUAP KHIRI K77180 THAILAND

##########
FORWARD EXPEDITION LIMITED HKG, MOSCOW REPRESENTATION,119002,RUSSIA,MOSCOW, KRIVOARBATSKIY PEREULOK, 13,BUILDING 2, 3RD FLOOR TEL:+7 (495)604-1200 EXT.106 FAX:+7 (495) 604-1
FORWARD EXPEDITION LIMITED HKG,MOSCOW  REPRESENTATION 119002, RUSSIA,MOSCOW, KRIVOARBATSKIY PEREULOK, 13, BUILDING 2, 3RD FLOOR TEL:+7(495)604-1200 EXT.106 FAX:+7(495)604-120
FORWARD EXPEDI

#### Klasteryzacja przy użyciu metryki dice
Pierwsze 100 lini z pliku result_dice.txt

In [13]:
with open("result_dice.txt", "r") as file:
    for _ in range(100):
        print(file.readline(), end="")

##########
WEISS ROHLIG CHINA CO.,LTD. UNIT 1712-1719,CORPORATE AVENUE 1 NO.222 HUBIN ROAD SHANGHAI 200021,CHINA TEL:+86-21-63406000 FAX:+86-21-63406858
WEISS-ROHLIG CHINA CO.,LTD. UNIT 1714-1719, CORPORATE AVENUE 1 NO. 222HU BIN ROAD 200021 SHANGHAI CHINATEL:86-021-63406000
WEISS ROHLIG CHINA CO LTD222 HU BIN ROADUNIT 1714-1719 CORPORATE AVENUE 1200021 SHANGHAI CHINA

##########
GOYA EN ESPANA, S.A. POLIGONO INDUSTRIAL LA RED CALLE 9 CARRETERA SEVILLA-MALAGA, KM 5.4 ALACALA DE GUADAIRA,SEVILLA, ESPANA 41500 C/O JOSEMANUEL CLEMENTE TEL: 3495-563-2032
GOYA EN ESPANA, SACTRA SEVILLA MALAGA  APDO CORREOS 6POLIGONO INDUSTRIAL LA RED KM 5,441500 ALCALA DE GUADAIRA
GOYA EN ESPANA POL.IND. "LA RED SUR" C/-9 N? 1 41500 ALCALA DE GUADAIRA, SEVILLA (SPAIN) TEL.: 955632032-955634134

##########
NINGBO JINMAO IMPORT AND EXPORT  CO.,LTD NO.173 ZHONGSHAN ROAD WEST  NINGBO,CHINA
SCAN GLOBAL LOGISTICS LTD O/B WINTIME IMPORT AND EXPORT CORPORATION  LIMITED OF ZHONGSHAN
ZHONGSHAN LONGDA IMPORT&EXPORT CO

Wnioski:
 - Dla każdej metryki użycie algorytmu AgglomerativeClustering z odpowiednio dobranym parametrem distance_threshold daje stosunkowo dobre wyniki
 - Metryka cosinusowa była najprostsza w implemetacji ponieważ wymagała jedynie pomnożenia macierzy rzadkiej przez jej transpozycję, przez co działa też najszybciej

## 5. Porównanie jakości wyników poprzez indeks Daviesa-Bouldina

In [14]:
with open("./lines.txt") as file:
    file_lines = file.readlines()
    file_lines = file_lines[:2000]

    n = len(file_lines)

    labels1 = cosine_clusterize(file_lines)
    labels2 = euclidean_clusterize(file_lines)
    labels3 = dice_clusterize(file_lines)
    labels4 = cosine_clusterize(file_lines, use_stopwords=False)
    labels5 = euclidean_clusterize(file_lines, use_stopwords=False)
    labels6 = dice_clusterize(file_lines, use_stopwords=False)

    X = get_document_term_matrix(file_lines, stopwords=get_stopwords(file_lines))
    db_score_cosine = round(davies_bouldin_score(X, labels1), 5)
    db_score_euclidean = round(davies_bouldin_score(X, labels2), 5)
    db_score_dice = round(davies_bouldin_score(X, labels3), 5)

    Y = get_document_term_matrix(file_lines, stopwords=None)
    db_score_cosine_no_stopwords = round(davies_bouldin_score(Y, labels4), 5)
    db_score_euclidean_no_stopwords = round(davies_bouldin_score(Y, labels5), 5)
    db_score_dice_no_stopwords = round(davies_bouldin_score(Y, labels6), 5)

    print(f"Obliczony indeks Daviesa-Boundina dla danej metryki: \n - cosinusowa: {' ' * 16} "
          f"{db_score_cosine} \n - euklidesowa:{' ' * 16} {db_score_euclidean} \n - dice:       {' ' * 16} {db_score_dice}"
          f"\n - cosinusowa (bez stoplisty):  {db_score_cosine_no_stopwords} \n - euklidesowa (bez stoplisty): "
          f"{db_score_euclidean_no_stopwords} \n - dice (bez stoplisty):        {db_score_dice_no_stopwords}")

Obliczony indeks Daviesa-Boundina dla danej metryki: 
 - cosinusowa:                  0.51292 
 - euklidesowa:                 0.49029 
 - dice:                        0.58649
 - cosinusowa (bez stoplisty):  0.52885 
 - euklidesowa (bez stoplisty): 0.5077 
 - dice (bez stoplisty):        0.61757


Wnioski:
 - Do obliczenia indeksu Daviesa-Boundina użyłem tylko pierwszych 2000 lini z pliku lines.txt ponieważ klasteryzacja na 6 sposobów + obliczenie samego indeksu 6 razy jest dość czasochłonne
 - Najlepsze wyniki uzyskałem dla metryki euklidesowej, cosinusowa jest niewiele gorsza a przy tym jej oblicznie jest dużo prostsze, natomiast metryka dice dała najgorsze wyniki

## 6. Pomysły na poprawę jakości klasteryzacji
Myślę, że gdyby zastosować bardziej złożony pre-processing, na przykład wyciągający z każdego wpisu jedynie użyteczne informacje, takie jak nazwa firmy, adres, numer telefonu, fax, email i następnie sprowadzić te dane do ustandaryzowanej formy to w połączeniu z metryką cosinusową lub euklidesową można by uzyskać jeszcze lepsze wyniki.